In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .getOrCreate()

spark

# Occupation

### Introduction:

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [ ]:
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"
spark.sparkContext.addFile(url)


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user).

### Step 3. Assign it to a variable called users.

In [ ]:
user = spark.read.csv(SparkFiles.get("u.user"), sep="|", header=True)
user.show()

### Step 4. Discover what is the mean age per occupation

In [ ]:
user.groupBy("occupation").agg(F.mean("age")).show()

In [ ]:
user.groupBy("occupation").agg(F.avg("age")).show()

### Step 5. Discover the Male ratio per occupation and sort it from the most to the least

In [ ]:
total_count = user.groupBy("occupation").count()
male_count = user.filter(user.gender == 'M').groupBy('occupation').count()

ratio = male_count.join(total_count, 'occupation', 'inner') \
    .withColumn('ratio', (male_count['count'] / total_count['count'])*100) \
    .orderBy('ratio', ascending=False)

ratio.show()

In [ ]:
total_people = user.groupBy("occupation").agg(F.count("user_id").alias("total"))
men_counts = user.where(F.col("gender") == "M").groupBy("occupation").agg(F.count("user_id").alias("men_count"))

result = men_counts.join(total_people, "occupation")
result = result.withColumn("male_ratio", (F.col("men_count") / F.col("total")) * 100)
result = result.sort(F.col("male_ratio").desc())
result.show()


### Step 6. For each occupation, calculate the minimum and maximum ages

In [ ]:
user.groupBy("occupation").agg(F.min("age"), F.max("age")).show()

### Step 7. For each combination of occupation and gender, calculate the mean age

In [ ]:
user.groupBy("occupation", "gender").agg(F.mean("age")).show()

### Step 8.  For each occupation present the percentage of women and men

In [ ]:

total_people = user.groupBy("occupation").agg(F.count("user_id").alias("total"))
gender_counts = user.groupBy("occupation", "gender").agg(F.count("user_id").alias("gender_count"))

result = gender_counts.join(total_people, "occupation")
result = result.withColumn("percentage", (F.col("gender_count") / F.col("total")) * 100)
result.show()